<a href="https://colab.research.google.com/github/catalinamagalvan/PySimMIBCI/blob/main/Example_generate_data_different_user_capabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Simulation of users with different brain modulation capabilities
This Google colab notebook illustrates how to generate MI-BCI data related to users with different capabilities to control a BCI.
For doing so, two strategies are proposed: 

1.   The first strategy employs different percentages of desynchronization in the $\alpha$ band to simulate the typical ERD in the contralateral hand motor area. In this example, subject S20 is simulated, considering a 20% of desynchronization. In the simulation of S20, all the trials have the same modulation level.
2. The second strategy consists in the inclusion of a certain proportion of failed MI trials, that is, trials without ERD in the corresponding area. In this example, subject SF30 is simulated, considering a 30% of trials without modulation. For SF30, the ideal percentage of desynchronization (50%) is used.

Notes:

*   These two strategies can also be combined to artificially generate users with different inclinations towards the control of a MI-BCI.
*   In order to see the classification results obtained for S20 and SF30, see the examples Example_classify_S20.ipynb and Example_classify_SF30.ipynb.



In [ ]:
!git clone https://github.com/catalinamagalvan/PySimMIBCI.git

Cloning into 'PySimMIBCI'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 67 (delta 34), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [ ]:
!pip install mne==1.3.0
!pip install fooof==1.0.0
!pip install colorednoise==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 KB 3.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from scipy.io import loadmat
import os
import mne
import numpy as np
from mne.datasets import fetch_fsaverage
import sys
sys.path.insert(1, os.path.join('PySimMIBCI', 'utils'))
from datasets import (load_and_epoch_OpenBMI_by_ftp, create_OpenBMI_info,
                      raw_from_OpenBMI_data_by_ftp)
from simulation import (set_up_source_forward, generate_when, generate_what,
                        generate_what_failed, generate_where)
from noise import add_aperiodic_activity, add_eye_movement
from plots import plot_raw_2_channels

# Set up general simulation parameters.

In [ ]:
# MI IDs
MI_tasks_IDs = {'MI/left': 1, 'MI/right': 2}
# Sample frequency
sfreq = 1000
# Number of trials
N_trials = 50
# MI trial duration in ms
MI_duration = 4500

Here, for both simulations user g from BCI competition IV dataset 1 is employed as the guide BCI-user. Thus, its aperiodic and aperiodic parameters are employed for the simulation.

In [ ]:
user_params = {'aperiodic_params': [-7.17, 1.14],
               'peak_params': {'G_precentral-lh': [9.54, 1, 2.52],
                               'G_precentral-rh': [10.13, 0.8, 1.62]}}

Generate suitable mne Info object for BCI competition IV dataset 1 data.

In [ ]:
info = create_BCIIV1_info()

#Simulation of subject S20
Parameters:
*   Percentage of ERD: 20%,
*   Percentage of failed trials: 0%.



In [ ]:
reduction = 0.2

Set up source space and compute forward solution.

Here, fsavarage head model is employed, which is a template brain based on a combination of 40 MRI scans of real brains.

In [ ]:
subject = 'fsaverage'
fwd, source_simulator = set_up_source_forward(subject, info)

Generate when, what and where information for the task-related component in right hand MI vs left hand MI scenario.

In [ ]:
events_info = {0: {'label': 'MI/left', 'duration': MI_duration},
               1: {'label': 'MI/right', 'duration': MI_duration}}
when = generate_when(events_info, N_trials, sfreq)
what = generate_what(MI_tasks_IDs.keys(), when, user_params, MI_duration,
                     sfreq, N_trials, reduction)
where = generate_where(subject)

Add task-related component to source simulator object.

In [ ]:
for label in what.keys():
    for task_ID, task in enumerate(MI_tasks_IDs.keys(), 1):
        when_task = when[np.where(when[:, 2] == task_ID)[0]]
        source_simulator.add_data(label=where[label],
                                  waveform=what[label][task],
                                  events=when_task)

Simulate raw data. 

Project the source time series to sensor space.
For doing so, the `mne.simulation.SourceSimulator` object is given directly to the `mne.simulation.simulate_raw` function.

In [ ]:
raw_sim_S20 = mne.simulation.simulate_raw(info, source_simulator, forward=fwd)
raw_sim_S20.set_eeg_reference(ref_channels='average')

Add non-task-related components:


*   Aperiodic activity (background noise) to raw simulated data Here, user-specific aperiodic parameters are employed.
*   Eog artifacts by MNE function.
*   Eye-movement artifacts.



In [ ]:
raw_sim_S20 = add_aperiodic_activity(raw_sim,
                                     exponent=user_params['aperiodic_params'][1],
                                     offset=user_params['aperiodic_params'][0])
mne.simulation.add_eog(raw_sim_S20)
add_eye_movement(raw_sim_S20)

Epoch data

In [ ]:
epochs_sim_S20 = mne.Epochs(raw_sim_S20, when, MI_tasks_IDs, tmin=0, tmax=4,
                            baseline=None)
epochs_sim_right_S20 = epochs_sim_S20['MI/right']
epochs_sim_left_S20 = epochs_sim_S20['MI/left']

#Generation of subject SF30
*   Percentage of ERD: 50% (ideal),
*   Percentage of failed trials: 30%.




In [ ]:
reduction = 0.5

Set up source space and compute forward solution.

Here, fsavarage head model is employed, which is a template brain based on a combination of 40 MRI scans of real brains.

In [ ]:
subject = 'fsaverage'
fwd, source_simulator = set_up_source_forward(subject, info)

Generate when, what and where information for the task-related component in right hand MI vs left hand MI scenario.

In [ ]:
events_info = {0: {'label': 'MI/left', 'duration': MI_duration},
               1: {'label': 'MI/right', 'duration': MI_duration}}
when = generate_when(events_info, N_trials, sfreq)
what = generate_what_failed(MI_tasks_IDs.keys(), when, user_params, MI_duration, sfreq,
                            N_trials, reduction, p_failed)
where = generate_where(subject)

Add task-related component to source simulator object.

In [ ]:
for label in what.keys():
    for task_ID, task in enumerate(MI_tasks_IDs.keys(), 1):
        when_task = when[np.where(when[:, 2] == task_ID)[0]]
        source_simulator.add_data(label=where[label],
                                  waveform=what[label][task],
                                  events=when_task)

Simulate raw data. 

Project the source time series to sensor space.
For doing so, the `mne.simulation.SourceSimulator` object is given directly to the `mne.simulation.simulate_raw` function.

In [ ]:
raw_sim_SF30 = mne.simulation.simulate_raw(info, source_simulator, forward=fwd)
raw_sim_SF30.set_eeg_reference(ref_channels='average')

#Add non-task-related components:

*   Aperiodic activity (background noise). Here, user-specific aperiodic parameters (exponent and offset) are employed.
*   Eog artifacts by [mne.simulation.add_eog](https://mne.tools/stable/generated/mne.simulation.add_eog.html) function.
*   Eye-movement artifacts.

In [ ]:
raw_sim_SF30 = add_aperiodic_activity(raw_sim_SF30,
                                      exponent=user_params['aperiodic_params'][1],
                                      offset=user_params['aperiodic_params'][0])
mne.simulation.add_eog(raw_sim_SF30)
add_eye_movement(raw_sim_SF30)

#Epoch data.

In [ ]:
epochs_sim_SF30 = mne.Epochs(raw_sim_SF30, when, MI_tasks_IDs, tmin=0, tmax=4,
                             baseline=None)
epochs_sim_right_SF30 = epochs_sim['MI/right']
epochs_sim_left_SF30 = epochs_sim['MI/left']